# load lib

In [15]:
import numpy as np
import pandas as pd

# load data

In [2]:
city_data = pd.read_csv('city_data.csv')
city_data

,city,x_co,y_co
0,1,0.3642,0.7770
1,2,0.7185,0.8312
2,3,0.0986,0.5891
3,4,0.2954,0.9606
4,5,0.5951,0.4647
5,6,0.6697,0.7657
6,7,0.4353,0.1709
7,8,0.2131,0.8349
8,9,0.3479,0.6984
9,10,0.4516,0.0488


In [3]:
city_position = city_data[['x_co','y_co']].to_numpy()

In [78]:
class GA(object):
    def __init__(self, DNA_size, cross_rate, mutation_rate, pop_size, ):
        ## get popuation 
        self.DNA_size = DNA_size
        self.cross_rate = cross_rate
        self.mutate_rate = mutation_rate
        self.pop_size = pop_size

        self.pop = np.vstack([np.random.permutation(DNA_size) for _ in range(pop_size)])

    def translateDNA(self, DNA, city_position):     # get cities' coord in order
        ## get dna
        line_x = np.empty_like(DNA, dtype=np.float64)
        line_y = np.empty_like(DNA, dtype=np.float64)
        for i, d in enumerate(DNA):
            city_coord = city_position[d]
            line_x[i, :] = city_coord[:, 0]
            line_y[i, :] = city_coord[:, 1]
        return line_x, line_y

    def get_distance(self, line_x, line_y):
        total_distance = np.empty((line_x.shape[0],), dtype=np.float64)
        for i, (xs, ys) in enumerate(zip(line_x, line_y)):
            total_distance[i] = np.sum(np.sqrt(np.square(np.diff(xs)) + np.square(np.diff(ys))))
        return total_distance
    
    def get_fitness(self, dis_list):                   #rank weighting selection
        dis_list = dict(list(enumerate(dis_list)))
        dis_list = dict(sorted(dis_list.items(), key = lambda kv:(kv[1], kv[0]),reverse=True))
        p = (np.arange(len(dis_list.values()))+1)/(np.sum(np.arange(len(dis_list.values()))+1))
        dis_list.update(zip(dis_list, p))
        dis_list = dict(sorted(dis_list.items()))
        fitness = dis_list.values()
        return list(fitness)    
    
    
    def select(self, fitness):
        idx = np.random.choice(np.arange(self.pop_size), size=self.pop_size, replace=True, p=fitness)
        return self.pop[idx]

    def crossover(self, parent, pop):
        if np.random.rand() < self.cross_rate:
            i_ = np.random.randint(0, self.pop_size, size=1)                        # select another individual from pop
            cross_points = np.random.randint(0, 2, self.DNA_size).astype(bool)   # choose crossover points
            keep_city = parent[~cross_points]                                       # find the city number
            swap_city = pop[i_, np.isin(pop[i_].ravel(), keep_city, invert=True)]
            parent[:] = np.concatenate((keep_city, swap_city))
        return parent

    def mutate(self, child):
        for point in range(self.DNA_size):
            if np.random.rand() < self.mutate_rate:
                swap_point = np.random.randint(0, self.DNA_size)
                swapA, swapB = child[point], child[swap_point]
                child[point], child[swap_point] = swapB, swapA
        return child

    def next_gen(self, fitness):
        pop = self.select(fitness)
        pop_copy = pop.copy()
        for parent in pop:  # for every parent
            child = self.crossover(parent, pop_copy)
            child = self.mutate(child)
            parent[:] = child
        self.pop = pop

 # Run GA

In [182]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%10 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 10 | best total_distance: 2.19963
best DNA: [7 1 5 8 2 6 3 4 9 0]
Gen: 20 | best total_distance: 2.05061
best DNA: [4 6 7 2 3 1 5 9 0 8]
Gen: 30 | best total_distance: 2.05061
best DNA: [9 6 4 5 1 2 0 8 7 3]
Gen: 40 | best total_distance: 1.94084
best DNA: [9 6 4 0 8 7 2 5 1 3]
Gen: 50 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 0 7 8 3 2]
Gen: 60 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 0 3 2 8 7]
Gen: 70 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 80 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 90 | best total_distance: 1.94084
best DNA: [2 6 4 5 1 0 8 3 7 9]
Gen: 100 | best total_distance: 1.94084
best DNA: [9 6 1 5 4 8 0 3 7 2]
Gen: 110 | best total_distance: 1.94084
best DNA: [6 9 4 5 1 8 0 3 7 2]
Gen: 120 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 130 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 140 | best total_distance: 1.94084
best DNA: [6 9 4 5 1 8 2 3 7 0]
G

# The shortest distance: 1.94084

# ===============================

# Try different parameters

 ## increase POP_SIZE, decrease N_GENERATIONS

In [113]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 500       #from 400 to 500
N_GENERATIONS = 300  #from 400 to 300

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%10 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 10 | best total_distance: 2.22656
best DNA: [9 6 3 7 2 8 0 5 1 4]
Gen: 20 | best total_distance: 2.03957
best DNA: [9 6 4 8 3 7 0 2 5 1]
Gen: 30 | best total_distance: 1.97692
best DNA: [9 4 8 2 7 0 3 1 5 6]
Gen: 40 | best total_distance: 1.94084
best DNA: [9 8 4 2 7 3 1 0 6 5]
Gen: 50 | best total_distance: 1.94084
best DNA: [0 8 2 7 3 1 5 4 6 9]
Gen: 60 | best total_distance: 1.94084
best DNA: [2 8 0 7 3 1 5 4 9 6]
Gen: 70 | best total_distance: 1.94084
best DNA: [2 7 5 0 8 3 1 4 6 9]
Gen: 80 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 5 1 4 6 9]
Gen: 90 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 100 | best total_distance: 1.94084
best DNA: [2 7 3 9 8 1 5 4 0 6]
Gen: 110 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 120 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 130 | best total_distance: 1.94084
best DNA: [7 1 4 2 3 0 8 5 6 9]
Gen: 140 | best total_distance: 1.94084
best DNA: [7 3 1 6 9 2 0 8 5 4]
G

In [115]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 700       #from 400 to 700
N_GENERATIONS = 100  #from 400 to 100

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%10 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 10 | best total_distance: 2.17502
best DNA: [9 6 4 8 0 2 7 5 1 3]
Gen: 20 | best total_distance: 2.07251
best DNA: [9 6 4 3 7 0 2 8 5 1]
Gen: 30 | best total_distance: 1.94084
best DNA: [9 6 4 8 2 0 7 3 1 5]
Gen: 40 | best total_distance: 1.94084
best DNA: [3 2 7 5 1 4 6 9 8 0]
Gen: 50 | best total_distance: 1.95640
best DNA: [9 4 1 2 6 7 8 5 3 0]
Gen: 60 | best total_distance: 1.95640
best DNA: [9 6 4 5 1 3 0 7 8 2]
Gen: 70 | best total_distance: 1.95640
best DNA: [9 6 1 0 4 3 7 5 8 2]
Gen: 80 | best total_distance: 1.95640
best DNA: [5 1 3 7 0 8 9 6 4 2]
Gen: 90 | best total_distance: 1.95640
best DNA: [9 6 4 5 1 3 7 0 8 2]
Gen: 100 | best total_distance: 1.95640
best DNA: [9 6 4 5 8 0 1 3 7 2]


 ## decrease POP_SIZE, increase N_GENERATIONS

In [119]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 300       #from 400 to 300
N_GENERATIONS = 500  #from 400 to 500

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 1.94084
best DNA: [6 4 5 7 9 1 8 0 3 2]
Gen: 100 | best total_distance: 1.94084
best DNA: [2 6 5 4 1 8 0 3 7 9]
Gen: 150 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 0 8 3 7 2]
Gen: 200 | best total_distance: 1.94084
best DNA: [9 6 8 3 7 2 4 1 5 0]
Gen: 250 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 7 0 3 8 2]
Gen: 300 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 350 | best total_distance: 1.94084
best DNA: [9 6 4 0 1 8 5 3 7 2]
Gen: 400 | best total_distance: 1.94084
best DNA: [1 3 8 9 6 4 5 0 7 2]
Gen: 450 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 500 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]


In [121]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 100       #from 400 to 100
N_GENERATIONS = 700  #from 400to 700

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.05805
best DNA: [9 6 4 1 5 0 3 8 2 7]
Gen: 100 | best total_distance: 1.94084
best DNA: [5 1 0 8 9 6 4 2 7 3]
Gen: 150 | best total_distance: 1.97692
best DNA: [9 6 4 5 1 0 2 3 7 8]
Gen: 200 | best total_distance: 1.94084
best DNA: [9 6 4 7 1 8 0 3 5 2]
Gen: 250 | best total_distance: 1.94084
best DNA: [9 6 4 8 0 7 5 2 1 3]
Gen: 300 | best total_distance: 1.94084
best DNA: [8 4 6 0 9 3 2 5 1 7]
Gen: 350 | best total_distance: 1.94084
best DNA: [9 4 1 5 0 8 3 2 6 7]
Gen: 400 | best total_distance: 1.94084
best DNA: [9 6 3 5 1 8 0 4 7 2]
Gen: 450 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 500 | best total_distance: 1.94084
best DNA: [6 3 8 5 1 7 2 9 4 0]
Gen: 550 | best total_distance: 1.94084
best DNA: [9 6 4 7 1 8 0 3 5 2]
Gen: 600 | best total_distance: 1.94084
best DNA: [3 0 8 2 7 9 6 4 5 1]
Gen: 650 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 700 | best total_distance: 1.94084
best DNA: [9 6 4 5 8 1 0 

In [125]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 50       #from 200 to 50
N_GENERATIONS = 750  #from 200 to 500

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.07012
best DNA: [8 7 3 0 2 1 5 4 6 9]
Gen: 100 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 150 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 200 | best total_distance: 1.94084
best DNA: [3 7 2 0 8 1 5 4 6 9]
Gen: 250 | best total_distance: 1.94084
best DNA: [2 7 3 0 5 8 1 4 6 9]
Gen: 300 | best total_distance: 1.95253
best DNA: [2 0 3 7 8 4 1 5 6 9]
Gen: 350 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 400 | best total_distance: 1.94084
best DNA: [2 9 1 5 7 3 0 8 4 6]
Gen: 450 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 500 | best total_distance: 1.94084
best DNA: [2 7 0 5 3 1 6 4 8 9]
Gen: 550 | best total_distance: 1.94084
best DNA: [5 9 6 2 7 3 0 8 1 4]
Gen: 600 | best total_distance: 1.94084
best DNA: [2 8 1 6 0 3 7 5 4 9]
Gen: 650 | best total_distance: 1.94084
best DNA: [2 3 0 1 6 5 9 7 8 4]
Gen: 700 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 

# increase/decrease CROSS_RATE

In [128]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.3  #from 0.2 to 0.3
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 1.95253
best DNA: [6 0 5 3 4 2 9 7 8 1]
Gen: 100 | best total_distance: 1.94084
best DNA: [9 6 4 1 5 8 0 3 7 2]
Gen: 150 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 200 | best total_distance: 1.94084
best DNA: [9 0 2 6 4 5 1 8 3 7]
Gen: 250 | best total_distance: 1.94084
best DNA: [9 6 4 8 1 3 5 2 7 0]
Gen: 300 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 350 | best total_distance: 1.94084
best DNA: [9 5 4 6 8 1 3 2 7 0]
Gen: 400 | best total_distance: 1.94084
best DNA: [5 8 4 7 1 9 6 2 3 0]


In [131]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.4  #from 0.2 to 0.4
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.19829
best DNA: [1 5 7 2 4 3 0 8 6 9]
Gen: 100 | best total_distance: 2.19635
best DNA: [9 0 6 4 1 8 5 3 7 2]
Gen: 150 | best total_distance: 2.15885
best DNA: [8 5 1 6 9 4 0 2 7 3]
Gen: 200 | best total_distance: 2.03957
best DNA: [2 3 5 7 8 0 9 6 4 1]
Gen: 250 | best total_distance: 2.36784
best DNA: [8 0 1 5 3 2 6 7 4 9]
Gen: 300 | best total_distance: 1.94084
best DNA: [4 6 9 2 0 8 1 3 7 5]
Gen: 350 | best total_distance: 2.10659
best DNA: [4 2 9 6 7 3 8 0 5 1]
Gen: 400 | best total_distance: 2.07767
best DNA: [5 8 9 0 3 7 2 4 1 6]


In [141]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.1  #from 0.2 to 0.1
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 1.95640
best DNA: [2 3 0 7 8 1 5 4 6 9]
Gen: 100 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 150 | best total_distance: 1.95640
best DNA: [4 8 0 7 3 1 5 2 6 9]
Gen: 200 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 250 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 6 5 4 1 9]
Gen: 300 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 350 | best total_distance: 1.94084
best DNA: [2 7 9 0 8 1 5 4 6 3]
Gen: 400 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 6 4 9]


In [142]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.05  #from 0.2 to 0.05
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 1.94084
best DNA: [9 6 4 1 5 8 0 3 7 2]
Gen: 100 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 150 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 200 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 250 | best total_distance: 1.94084
best DNA: [9 6 4 1 5 8 0 3 7 2]
Gen: 300 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 350 | best total_distance: 1.94084
best DNA: [9 6 4 5 1 8 0 3 7 2]
Gen: 400 | best total_distance: 1.94084
best DNA: [4 5 1 8 0 7 9 6 3 2]


In [143]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.01  #from 0.2 to 0.05
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 100 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 6 4 9]
Gen: 150 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 5 1 4 6 9]
Gen: 200 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 250 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 300 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 350 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 400 | best total_distance: 1.95640
best DNA: [2 8 0 7 1 3 5 4 6 9]


In [148]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0. #from 0.2 to 0.0
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.18737
best DNA: [2 7 3 0 8 6 9 4 5 1]
Gen: 100 | best total_distance: 2.18737
best DNA: [2 8 3 0 7 6 9 4 5 1]
Gen: 150 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 200 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 250 | best total_distance: 1.94084
best DNA: [2 7 3 0 4 1 5 8 6 9]
Gen: 300 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 350 | best total_distance: 1.94084
best DNA: [2 7 3 0 8 1 5 4 6 9]
Gen: 400 | best total_distance: 1.94084
best DNA: [3 7 2 0 8 1 5 4 6 9]


# increase/decrease MUTATE_RATE


In [150]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.07 #from 0.03 to 0.07
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.16502
best DNA: [7 8 4 3 0 1 2 5 9 6]
Gen: 100 | best total_distance: 2.27628
best DNA: [6 9 2 3 7 0 5 1 4 8]
Gen: 150 | best total_distance: 2.20755
best DNA: [2 7 0 8 1 5 9 6 4 3]
Gen: 200 | best total_distance: 2.08663
best DNA: [2 6 4 8 5 0 3 7 9 1]
Gen: 250 | best total_distance: 2.14403
best DNA: [2 1 5 0 7 3 8 4 6 9]
Gen: 300 | best total_distance: 2.05805
best DNA: [4 0 2 6 9 3 1 7 8 5]
Gen: 350 | best total_distance: 1.97692
best DNA: [3 7 8 2 0 1 5 4 6 9]
Gen: 400 | best total_distance: 2.03120
best DNA: [6 9 4 8 7 3 1 5 0 2]


In [151]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.1 #from 0.03 to 0.1
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.21121
best DNA: [3 8 0 5 7 2 6 9 4 1]
Gen: 100 | best total_distance: 2.16440
best DNA: [7 8 4 9 6 3 5 1 0 2]
Gen: 150 | best total_distance: 2.31134
best DNA: [0 5 3 9 6 2 7 8 4 1]
Gen: 200 | best total_distance: 2.00562
best DNA: [2 0 7 6 8 1 3 9 4 5]
Gen: 250 | best total_distance: 2.14258
best DNA: [6 4 2 0 8 7 3 1 5 9]
Gen: 300 | best total_distance: 2.28053
best DNA: [6 4 9 0 2 7 8 1 3 5]
Gen: 350 | best total_distance: 2.30010
best DNA: [9 2 7 4 8 1 3 5 6 0]
Gen: 400 | best total_distance: 2.25299
best DNA: [0 3 4 9 6 8 2 7 5 1]


In [155]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.02 #from 0.03 to 0.02
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 1.95640
best DNA: [2 9 3 1 4 6 8 7 5 0]
Gen: 100 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 150 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 200 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 250 | best total_distance: 1.95640
best DNA: [2 8 0 1 3 6 5 4 7 9]
Gen: 300 | best total_distance: 1.95640
best DNA: [2 7 3 6 9 8 0 1 5 4]
Gen: 350 | best total_distance: 1.95640
best DNA: [2 8 0 7 3 1 5 4 6 9]
Gen: 400 | best total_distance: 1.95640
best DNA: [3 8 0 7 2 1 5 4 6 9]


In [162]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.0 #from 0.03 to 0.0
POP_SIZE = 400
N_GENERATIONS = 400


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%50 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 50 | best total_distance: 2.07488
best DNA: [5 1 0 3 7 2 8 4 6 9]
Gen: 100 | best total_distance: 2.07488
best DNA: [1 5 0 3 7 2 8 4 6 9]
Gen: 150 | best total_distance: 2.07488
best DNA: [1 5 0 3 7 2 8 4 6 9]
Gen: 200 | best total_distance: 2.07488
best DNA: [1 5 0 3 7 2 8 4 6 9]
Gen: 250 | best total_distance: 2.07488
best DNA: [1 5 0 3 7 2 8 4 6 9]
Gen: 300 | best total_distance: 2.07488
best DNA: [1 5 0 3 7 2 8 4 6 9]
Gen: 350 | best total_distance: 2.07488
best DNA: [1 5 0 3 7 2 8 4 6 9]
Gen: 400 | best total_distance: 2.07488
best DNA: [5 0 3 2 8 4 6 9 1 7]


# ==========================

# Introduce more cities 

In [175]:
city_data = pd.read_csv('city_data_13.csv')
city_data

,city,x_co,y_co
0,0,0.3642,0.7770
1,1,0.7185,0.8312
2,2,0.0986,0.5891
3,3,0.2954,0.9606
4,4,0.5951,0.4647
5,5,0.6697,0.7657
6,6,0.4353,0.1709
7,7,0.2131,0.8349
8,8,0.3479,0.6984
9,9,0.4516,0.0488


In [176]:
city_position = city_data[['x_co','y_co']].to_numpy()

In [185]:
N_CITIES = 13  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 600
N_GENERATIONS = 500


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%20 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 20 | best total_distance: 3.60163
best DNA: [ 4  1  3  2  8  7  5  0 10  6 11  9 12]
Gen: 40 | best total_distance: 3.31030
best DNA: [12  4  1  5  7  0  8  3  2 10  9  6 11]
Gen: 60 | best total_distance: 3.21815
best DNA: [ 2  8  0  3  7  5  1  4 12  6  9 10 11]
Gen: 80 | best total_distance: 3.04657
best DNA: [12  4  5  1  0  3  8  7  2 10  6  9 11]
Gen: 100 | best total_distance: 2.99040
best DNA: [12  4  1  5  0  7  3  8  2 10  6  9 11]
Gen: 120 | best total_distance: 2.99040
best DNA: [12  4  1  8  2  5  3  7  0 10  6  9 11]
Gen: 140 | best total_distance: 2.99040
best DNA: [12  4  5  7  3  1  0  8  2 10  6  9 11]
Gen: 160 | best total_distance: 2.99040
best DNA: [12  4  5  1  3  7  0  8  2 10  6  9 11]
Gen: 180 | best total_distance: 2.99040
best DNA: [ 6  4  5  1  3  7  0  8  2 10 12  9 11]
Gen: 200 | best total_distance: 2.99040
best DNA: [12  4  5  1  3  7  0  8  2 10  6  9 11]
Gen: 220 | best total_distance: 2.99040
best DNA: [12 10  5  1  3  7  0  8  2  4  6  9 11]
Gen

# =====================

# Code testing

In [6]:
N_CITIES = 10  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 400
N_GENERATIONS = 400


# get popuation , dna

In [7]:
def get_pop( N_CITIES , POP_SIZE ):
    return np.vstack([np.random.permutation(N_CITIES) for _ in range(POP_SIZE)])

pop = get_pop( N_CITIES , POP_SIZE )

In [8]:
pop

array([[8, 3, 2, ..., 5, 1, 6],
       [3, 7, 2, ..., 5, 8, 0],
       [8, 7, 0, ..., 6, 5, 3],
       ...,
       [4, 2, 0, ..., 3, 5, 6],
       [4, 7, 0, ..., 1, 6, 3],
       [8, 7, 0, ..., 3, 5, 2]])

In [9]:
def get_dna( pop , city_position ):   
    line_x = np.empty_like(pop, dtype=np.float64)
    line_y = np.empty_like(pop, dtype=np.float64)
    for i, d in enumerate(pop):
                city_coord = city_position[d]
                line_x[i, :] = city_coord[:, 0]
                line_y[i, :] = city_coord[:, 1]
                
    return line_x, line_y


line_x, line_y = get_dna(pop ,city_position)

In [10]:
line_x

array([[0.3479, 0.2954, 0.0986, ..., 0.6697, 0.7185, 0.4353],
       [0.2954, 0.2131, 0.0986, ..., 0.6697, 0.3479, 0.3642],
       [0.3479, 0.2131, 0.3642, ..., 0.4353, 0.6697, 0.2954],
       ...,
       [0.5951, 0.0986, 0.3642, ..., 0.2954, 0.6697, 0.4353],
       [0.5951, 0.2131, 0.3642, ..., 0.7185, 0.4353, 0.2954],
       [0.3479, 0.2131, 0.3642, ..., 0.2954, 0.6697, 0.0986]])

In [11]:
line_y

array([[0.6984, 0.9606, 0.5891, ..., 0.7657, 0.8312, 0.1709],
       [0.9606, 0.8349, 0.5891, ..., 0.7657, 0.6984, 0.777 ],
       [0.6984, 0.8349, 0.777 , ..., 0.1709, 0.7657, 0.9606],
       ...,
       [0.4647, 0.5891, 0.777 , ..., 0.9606, 0.7657, 0.1709],
       [0.4647, 0.8349, 0.777 , ..., 0.8312, 0.1709, 0.9606],
       [0.6984, 0.8349, 0.777 , ..., 0.9606, 0.7657, 0.5891]])

# get distance, fitness

In [168]:
def get_distance(line_x, line_y):
    total_distance = np.empty((line_x.shape[0],), dtype=np.float64)
    for i, (xs, ys) in enumerate(zip(line_x, line_y)):
        total_distance[i] = np.sum(np.sqrt(np.square(np.diff(xs)) + np.square(np.diff(ys))))
    return total_distance 
        
    
    
def get_fitness(dis_list):
    dis_list = dict(list(enumerate(dis_list)))
    dis_list = dict(sorted(dis_list.items(), key = lambda kv:(kv[1], kv[0]),reverse=True))
    p = (np.arange(len(dis_list.values()))+1)/(np.sum(np.arange(len(dis_list.values()))+1))
    dis_list.update(zip(dis_list, p))
    dis_list = dict(sorted(dis_list.items()))
    fitness = dis_list.values()
    return list(fitness)
    
    
total_distance = get_distance(line_x, line_y)    
fitness = get_fitness(total_distance)

In [169]:
total_distance

array([4.91220969, 4.50375698, 3.7185235 , 3.33135492, 4.90750843,
       4.69110921, 2.88840709, 4.44971159, 3.85561328, 2.96803428,
       4.64660537, 4.93191676, 4.03721722, 4.60247382, 3.98730211,
       4.53805502, 3.58474865, 4.35526217, 4.73708322, 4.20104865,
       3.72991617, 4.70225051, 4.40613151, 3.79255469, 4.03575344,
       4.50629764, 3.88892535, 3.98423501, 4.25864106, 3.56433785,
       5.23643755, 3.75608874, 4.2690561 , 3.12994611, 3.83146137,
       5.04197138, 4.72534779, 4.6459332 , 4.37349991, 4.82876686,
       4.48659987, 4.10625245, 4.1293333 , 3.931129  , 4.4032567 ,
       4.37755013, 4.17041283, 5.24330633, 4.70642286, 4.57252692,
       4.53819236, 3.77092768, 3.69138627, 3.54600475, 3.66830524,
       3.94441973, 4.49221718, 5.22750424, 4.97639054, 4.25422296,
       3.62352147, 3.52716413, 4.86354923, 4.76070155, 5.08100151,
       4.07403494, 4.16032759, 4.92583404, 4.05694706, 3.20136601,
       3.42010758, 4.07211432, 4.7486911 , 4.12328045, 4.04036

In [170]:
fitness

[0.0004488778054862843,
 0.001546134663341646,
 0.004064837905236908,
 0.004738154613466334,
 0.0004738154613466334,
 0.0009351620947630922,
 0.004912718204488778,
 0.0018079800498753118,
 0.003778054862842893,
 0.004875311720698255,
 0.000997506234413965,
 0.0003740648379052369,
 0.0032418952618453864,
 0.00114713216957606,
 0.0034164588528678304,
 0.001396508728179551,
 0.0043890274314214465,
 0.0020698254364089775,
 0.0007605985037406484,
 0.0025561097256857853,
 0.004039900249376558,
 0.0008977556109725686,
 0.00185785536159601,
 0.003927680798004988,
 0.003254364089775561,
 0.0015211970074812968,
 0.0036907730673316707,
 0.0034413965087281796,
 0.002418952618453865,
 0.00442643391521197,
 6.234413965087281e-05,
 0.00401496259351621,
 0.0023815461346633417,
 0.00483790523690773,
 0.0038279301745635912,
 0.00021197007481296758,
 0.0008104738154613466,
 0.0010099750623441396,
 0.0020448877805486283,
 0.0005860349127182044,
 0.0016458852867830423,
 0.0028802992518703243,
 0.0027930174

#  select , crossover , mutate

In [171]:
def select(fitness , pop , POP_SIZE):
        idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p = fitness)
        return pop[idx]

def crossover(parent, pop , CROSS_RATE , POP_SIZE , N_CITIES):
        if np.random.rand() < CROSS_RATE:
            i_ = np.random.randint(0, POP_SIZE, size=1)                        # select another individual from pop
            cross_points = np.random.randint(0, 2, N_CITIES).astype(bool)   # choose crossover points
            keep_city = parent[~cross_points]                                       # find the city number
            swap_city = pop[i_, np.isin(pop[i_].ravel(), keep_city, invert=True)]
            parent[:] = np.concatenate((keep_city, swap_city))
        return parent
    
    
def mutate(child , MUTATE_RATE ,N_CITIES ):
    for point in range(N_CITIES):
        if np.random.rand() < MUTATE_RATE:
            swap_point = np.random.randint(0, N_CITIES)
            swapA, swapB = child[point], child[swap_point]
            child[point], child[swap_point] = swapB, swapA
    return child

In [172]:
pop

array([[1, 9, 8, ..., 0, 4, 7],
       [7, 2, 9, ..., 1, 4, 3],
       [9, 2, 8, ..., 7, 5, 3],
       ...,
       [3, 5, 6, ..., 4, 9, 0],
       [4, 0, 8, ..., 3, 2, 5],
       [4, 9, 7, ..., 8, 6, 5]])

In [173]:
def next_gen(fitness ,pop, CROSS_RATE ,MUTATE_RATE, POP_SIZE , N_CITIES):
    pop = select(fitness , pop , POP_SIZE)
    pop_copy = pop.copy()
    for parent in pop:  # for every parent
        child = crossover(parent, pop_copy ,CROSS_RATE , POP_SIZE , N_CITIES)
        child = mutate(child , MUTATE_RATE ,N_CITIES )
        parent[:] = child
    return pop
        

pop = next_gen(fitness = fitness, pop = pop , POP_SIZE = POP_SIZE, CROSS_RATE =CROSS_RATE , MUTATE_RATE = MUTATE_RATE , N_CITIES = N_CITIES)

In [174]:
pop

array([[9, 6, 7, ..., 1, 8, 4],
       [2, 0, 3, ..., 4, 7, 8],
       [3, 5, 2, ..., 6, 8, 4],
       ...,
       [8, 2, 6, ..., 4, 5, 9],
       [8, 2, 6, ..., 4, 5, 9],
       [7, 4, 5, ..., 3, 6, 1]])

In [17]:
np.square(5-2)

9

In [14]:
[1,2,3,4,5][-1]

5